# Generate Datasets using DC approximation for Power Grid usecase
This notebook allows to generate datasets for training and evaluation of augmented simulators using DC solver. In this case, the environment parameters should be modified to let the DC approximation be activate. Furthermore, we require to store the information concerning the admittance matrices via DC specific functions, that are implemented and integrated in various LIPS modules.

In [53]:
import pathlib

LIPS_PATH = pathlib.Path().resolve().parent.parent
CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_case14_sandbox.ini"
DATA_PATH = LIPS_PATH / "reference_data" / "powergrid" / "l2rpn_case14_sandbox" / "DC"
LOG_PATH = LIPS_PATH / "lips_logs.log"

In [54]:
if not DATA_PATH.exists():
    DATA_PATH.mkdir(mode=511, parents=False)

In [56]:
from lips.benchmark.powergridBenchmark import PowerGridBenchmark

benchmark4 = PowerGridBenchmark(benchmark_path=DATA_PATH,
                                benchmark_name="DoNothing",
                                load_data_set=False,
                                config_path=CONFIG_PATH,
                                log_path=LOG_PATH)

In [59]:
benchmark4.generate(nb_sample_train=int(1e5),
                    nb_sample_val=int(1e4),
                    nb_sample_test=int(1e4),
                    nb_sample_test_ood_topo=int(1e4),
                    do_store_physics=True,
                    is_dc=True
                   )

100%|██████████| 10000/10000 [01:04<00:00, 154.27it/s]


### Error illustration 
An error occurs when using Benchmark3 that presents topological changes. The error source is related to when getting admittance matrix by `get_dcYbus()` function.

In [ ]:
from lips.benchmark.powergridBenchmark import PowerGridBenchmark

benchmark3 = PowerGridBenchmark(benchmark_path=DATA_PATH,
                                benchmark_name="Benchmark3",
                                load_data_set=False,
                                config_path=CONFIG_PATH,
                                log_path=LOG_PATH)

In [ ]:
benchmark3.generate(nb_sample_train=int(1e5),
                    nb_sample_val=int(1e4),
                    nb_sample_test=int(1e4),
                    nb_sample_test_ood_topo=int(1e4),
                    do_store_physics=True,
                    is_dc=True
                   )

In [9]:
YBus = benchmark3.training_simulator._simulator.backend._grid.get_dcYbus().todense()

In [11]:
env = benchmark3.training_simulator._simulator
obs = env.get_obs()

It can be seen that the dimension of unique bus bars is 14 however the returned admittance matrix for the same observation is 15x15 with an empty row and column.

In [12]:
nb_bus, unique_bus, bus_or, bus_ex = obs._aux_fun_get_bus()
n_bus_bars = obs._obs_env.n_sub * 2
print(n_bus_bars)
print(unique_bus)
print(len(unique_bus))

28
[ 0  1  2  3  4  5  6  7  9 10 11 12 13 22]
14


In [ ]:
# Here is the row of admittance matrix which contains only zeros
YBus[8]

matrix([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

There is an empty row and column in admittance matrix return using `get_dcYbus` function

In [18]:
import numpy as np
np.where(~YBus.any(axis=1))[0]

array([8])

Removing the empty line and column from admittance matrix

In [40]:
indices_rows = np.array(YBus.any(axis=1).flatten()).reshape(-1,)
indices_cols = np.array(YBus.any(axis=0).flatten()).reshape(-1,)

In [52]:
YBus[np.ix_(indices_rows, indices_cols)].shape

(14, 14)

## Generate one step without using LIPS

In [47]:
import grid2op
from grid2op.Agent import DoNothingAgent
from lightsim2grid import LightSimBackend
from lips.dataset.utils.powergrid_utils import get_kwargs_simulator_scenario

In [42]:
kwargs_simulator = get_kwargs_simulator_scenario(benchmark3.config)
kwargs_simulator

{'dataset': 'l2rpn_case14_sandbox',
 'param': <grid2op.Parameters.Parameters at 0x7f9528b728b0>,
 'data_feeding_kwargs': {'gridvalueClass': grid2op.Chronics.gridStateFromFile.GridStateFromFile},
 'action_class': grid2op.Action.PlayableAction.PlayableAction,
 'opponent_init_budget': 0,
 'opponent_action_class': grid2op.Action.DontAct.DontAct,
 'backend': <lightsim2grid.lightSimBackend.LightSimBackend at 0x7f9528bab700>}

In [44]:
env = grid2op.make(**kwargs_simulator)

In [46]:
env.parameters.to_dict()

{'NO_OVERFLOW_DISCONNECTION': True,
 'IGNORE_MIN_UP_DOWN_TIME': True,
 'ALLOW_DISPATCH_GEN_SWITCH_OFF': True,
 'LIMIT_INFEASIBLE_CURTAILMENT_STORAGE_ACTION': False,
 'NB_TIMESTEP_OVERFLOW_ALLOWED': 2,
 'NB_TIMESTEP_RECONNECTION': 10,
 'HARD_OVERFLOW_THRESHOLD': 2.0,
 'ENV_DC': True,
 'FORECAST_DC': False,
 'MAX_SUB_CHANGED': 999999,
 'MAX_LINE_STATUS_CHANGED': 999999,
 'NB_TIMESTEP_COOLDOWN_LINE': 0,
 'NB_TIMESTEP_COOLDOWN_SUB': 0,
 'INIT_STORAGE_CAPACITY': 0.5,
 'ACTIVATE_STORAGE_LOSS': True,
 'ALARM_BEST_TIME': 12,
 'ALARM_WINDOW_SIZE': 12,
 'MAX_SIMULATE_PER_STEP': -1,
 'MAX_SIMULATE_PER_EPISODE': -1}

In [51]:
agent = DoNothingAgent(env.action_space)
action = agent.act(None, None)
print(action)

This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT modify any storage capacity
	 - NOT perform any curtailment
	 - NOT force any line status
	 - NOT switch any line status
	 - NOT switch anything in the topology
	 - NOT force any particular bus configuration


In [52]:
obs = env.reset()
obs, *_ = env.step(action)

In [60]:
env.backend._grid.get_dcYbus().todense().shape

(14, 14)